In [ ]:
%pip install finta
%pip install yfinance
%pip install backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import requests
import json
from finta import TA
import pandas as pd

In [ ]:
url = 'https://api.pro.coinbase.com/products/ADA-USD/candles'
params = {'start': '2023-04-07T00:00:00.000Z', 'end': '2023-04-18T23:59:59.999Z', 'granularity': 3600}
response = requests.get(url, params=params)
data = json.loads(response.text)

df = pd.DataFrame(data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Date'] = pd.to_datetime(df['Date'], unit='s')
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df

In [69]:
class SmaCrossWithRSI(Strategy):
    # Initialize variables
    short_sma = 20
    long_sma = 50
    atr_length = 14
    rsi_length = 14

    # Initialize the strategy. Instances are stored as attributes in our SmaCrossWithRSI object. 
    def init(self):
        # Creates pandas series of the price data for later use. 
        open = pd.Series(self.data.Open)
        close = pd.Series(self.data.Close)
        high = pd.Series(self.data.High)
        low = pd.Series(self.data.Low)
        volume = pd.Series(self.data.Volume)

        # Creates df for the ohlcv data to calculate ATR and RSI.
        ohlcv = pd.DataFrame({'open': open, 'high': high, 'low': low, 'close': close, 'volume': volume})

        # Create instance of ATR and RSI indicators from finta, and the native SMA indicator from bt.
        self.atr = self.I(TA.ATR, ohlcv, self.atr_length)
        self.rsi = self.I(TA.RSI, ohlcv, self.rsi_length)
        self.sma1 = self.I(SMA, close, self.short_sma)
        self.sma2 = self.I(SMA, close, self.long_sma)

    # Logic for trade entrance and exit
    def next(self):
        # Price set to the candles 'close' value, used for calculating stop loss and take profit targets.
        price = self.data.Close

        # ATR and RSI for the latest candle.
        ATR = self.atr[-1]
        RSI = self.rsi[-1]

        # If sma1 crosses over sma2, and RSI > 50, then enter a long position with the desired target logic.
        if crossover(self.sma1, self.sma2) and RSI > 50:
            stop_loss = price - (1.1 * ATR)
            take_profit = price + (2 * ATR)
            self.buy(sl=stop_loss, tp=take_profit)

In [71]:
bt = Backtest(df, SmaCrossWithRSI, cash=100000, commission=0.002, exclusive_orders=True)

strategy_trade_results = bt.run()
strategy_trade_results

Start                     2023-04-07 00:00:00
End                       2023-04-18 23:00:00
Duration                     11 days 23:00:00
Exposure Time [%]                    5.208333
Equity Final [$]                100006.898474
Equity Peak [$]                  100096.74406
Return [%]                           0.006898
Buy & Hold Return [%]                8.104244
Return (Ann.) [%]                    0.210042
Volatility (Ann.) [%]                5.674852
Sharpe Ratio                         0.037013
Sortino Ratio                        0.072673
Calmar Ratio                         0.240399
Max. Drawdown [%]                   -0.873721
Avg. Drawdown [%]                   -0.476793
Max. Drawdown Duration        8 days 23:00:00
Avg. Drawdown Duration        4 days 13:00:00
# Trades                                    3
Win Rate [%]                        33.333333
Best Trade [%]                       0.867063
Worst Trade [%]                     -0.485102
Avg. Trade [%]                    

In [ ]:
bt.plot()